# Sampta Experience II

In [ ]:
import numpy as np
import torch
import matplotlib.pyplot as plt

import torch_harmonics as th
import spherical_inr as sph
from sklearn.preprocessing import MinMaxScaler

In [ ]:
def compute_snr(image, noisy_image):
    """
    Computes the Signal-to-Noise Ratio (SNR) between two images.
    """
    
    signal_power = np.mean(np.square(image))
    noise_power = np.mean(np.square(image - noisy_image))
    snr = 10 * np.log10(signal_power / noise_power)
    
    return snr

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")